# **RFM분석과 Retention분석**
***

## 질문 만들기
- 시간의 흐름에 따라 매출, 주문고객수, 주문단가의 추이는?
- 리텐션 분석: 시간의 흐름에 따라 고객은 얼마나 남고 이탈?
- RFM 분석: 고객의 행동에 따라 고객을 유형화 해보자

- **리텐션 분석이란?**
    - 유저가 제품을 사용한 이후 일정 기간 지난 시점에 제품을 계속 사용하고 있는지 유저의 잔존과 이탈을 분석하는 방법
    - DAY0에 방문한 유저 중 DAY1에 방문한 비율을 DAY단위 리텐션이라 할 수 있음
    - 일반적으로 리텐션이 높으면 유저가 서비스를 주기적으로 사용한다는 뜻
    - 유저의 참여와 충성도를 높이기 위한 전략을 정하는데 중요한 지표

- **RFM분석이란?**
    - Recency, Frequency, Monetary를 기반으로 고객을 유형화하는 방법
        - Recency(최근성): 고객이 얼마나 최근에 구매를 하였는지
        - Frequency(빈도): 고객이 얼마나 자주 구매하였는지
        - Monetary(금액): 고객이 구매한 금액
    - 고객 유형을 세분화하여 맞춤형 전략을 구상할 수 있다
        - 예: 총 구매금액은 낮지만 자주 방문하는 유저 VS 최근에 큰 금액을 구매했지만 자주 방문하지는 않았던 유저
- **여기서 데이터 분석가가 분석하고 싶은 지표(구매, 장바구니담기, 웹사이트 방문)등으로 지표를 바꿀 수도 있다.**

## **1. 데이터 전처리**

In [36]:
import pandas as pd
import plotly.express as px

In [37]:
df = pd.read_csv(r'.\실습3_ 유통 데이터를 활용한 리텐션과 RFM 분석\ecommerce_data.csv',
                 encoding='utf8')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


- **결측치 제거**
- 고객에 대한 분석을 할 예정이므로 CustomerID가 없는 행은 제거


In [39]:
df = df.dropna(subset='CustomerID',axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


- **날짜형태 데이터타입을 변경**
- **CustomerID를 문자형식으로 변경**

In [40]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'],format='%m/%d/%Y %H:%M')
df['CustomerID'] = df['CustomerID'].astype('int').astype('str')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [41]:
df['date_ymd'] = df['InvoiceDate'].dt.date
df['date_ymd'] = df['date_ymd'].astype('datetime64[ns]')
df['year'] = df['date_ymd'].astype('datetime64[ns]').dt.year
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_ymd,year
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12-01,2010
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12-01,2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010


- 매출 칼럼 추가하기 (수량 * 가격)

In [42]:
df['amount'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_ymd,year,amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12-01,2010,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12-01,2010,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34


- Quantity가 음수인 것은 취소를 뜻하므로 제외하기

In [43]:
df[df['Quantity'] == 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_ymd,year,amount


In [44]:
df = df.loc[df['Quantity'] > 0,:]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  object        
 7   Country      397924 non-null  object        
 8   date_ymd     397924 non-null  datetime64[ns]
 9   year         397924 non-null  int32         
 10  amount       397924 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int32(1), int64(1), object(5)
memory usage: 34.9+ MB


## **2. 분석**
- 일자별 매출 추이는?
    - 조금씩 우상향
- 일자별 주문 고객수는(Unique한)?
- 일자별 주문단가의 추이는?

In [45]:
amount_by_day = df.groupby(['date_ymd'],as_index=False)[['amount']].agg('sum')
fig = px.line(amount_by_day,x='date_ymd',y='amount',
              hover_data={'date_ymd':'|%Y년 %m월 %d일','amount':':.2f'},
              labels={'date_ymd':'날짜','amount':'매출'},
              )
fig.update_traces(line={'color':'red'})
fig.update_layout(xaxis_title='날짜',yaxis_title='매출',title='일별 매출 추이',
                plot_bgcolor='lightblue',paper_bgcolor='lightgray')

c:\Users\kd010\miniconda3\envs\rapids\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [46]:
unique_customers_by_day = df.groupby('date_ymd',as_index=False)[['CustomerID']].nunique().rename(columns={'CustomerID':'주문고객수'})
fig = px.line(unique_customers_by_day,x='date_ymd',y='주문고객수',
        hover_data={'date_ymd':'|%Y년 %m월 %d일'},
                    labels = {'date_ymd':'날짜'})
fig.update_layout(title='일별 주문고객수 추이',paper_bgcolor = 'lightgray',plot_bgcolor = 'lightblue')
fig.update_traces(line={'color':'red'})

c:\Users\kd010\miniconda3\envs\rapids\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



- 주문단가(주문 1건당 매출액)

In [47]:
revenue_by_date_orders = df.groupby('date_ymd',as_index=False).agg({'amount':'sum','InvoiceNo':'nunique'}).rename(columns={'InvoiceNo':'주문건수'})
revenue_by_date_orders['주문단가'] = revenue_by_date_orders['amount'] / revenue_by_date_orders['주문건수']
fig = px.line(revenue_by_date_orders,x='date_ymd',
        y='주문단가',hover_data={'date_ymd':'|%Y년 %m월 %d일'},labels={'date_ymd':'날짜'})
fig.update_layout(title='일별 주문단가',plot_bgcolor='lightblue',paper_bgcolor='lightgray')
fig.update_traces(line=dict(color='red'))

c:\Users\kd010\miniconda3\envs\rapids\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [48]:
revenue_by_date_orders

,date_ymd,amount,주문건수,주문단가
0,2010-12-01,46376.49,121,383.276777
1,2010-12-02,47316.53,137,345.376131
2,2010-12-03,23921.71,57,419.679123
3,2010-12-05,31771.60,87,365.190805
4,2010-12-06,31215.64,94,332.081277
...,...,...,...,...
300,2011-12-05,58202.21,116,501.743190
301,2011-12-06,46144.04,110,419.491273
302,2011-12-07,69354.21,104,666.867404
303,2011-12-08,50519.41,113,447.074425


## 2.**리텐션분석**
***
- **월별로 고객은 얼마나 남고 얼마나 이탈했는가?**

- 월별 RFM분석을 할 것이기 때문에 월별 칼럼을 만들어 준다

In [49]:
df['date_ym'] = df['date_ymd'].dt.to_period('M')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_ymd,year,amount,date_ym
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12-01,2010,15.30,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12-01,2010,22.00,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34,2010-12
...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,2011-12-09,2011,10.20,2011-12
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,2011-12-09,2011,12.60,2011-12
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,2011-12-09,2011,16.60,2011-12
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,2011-12-09,2011,16.60,2011-12


In [50]:
print(max(df['date_ym']))
print(min(df['date_ym']))

2011-12
2010-12


- 2010년 12월부터 2011년 12월까지의 retention 분석을 할건데
- 2011년 12월데이터를 시작일로 보면 당연히 매우 낮을 수밖에 없으니 input에서는 제외

In [51]:
retention_base = df[df['date_ym']<'2011-12']
retention_base

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_ymd,year,amount,date_ym
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12-01,2010,15.30,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12-01,2010,22.00,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34,2010-12
...,...,...,...,...,...,...,...,...,...,...,...,...
516364,579885,85034C,3 ROSE MORRIS BOXED CANDLES,4,2011-11-30 17:37:00,1.25,15444,United Kingdom,2011-11-30,2011,5.00,2011-11
516365,579885,21742,LARGE ROUND WICKER PLATTER,2,2011-11-30 17:37:00,5.95,15444,United Kingdom,2011-11-30,2011,11.90,2011-11
516366,579885,23084,RABBIT NIGHT LIGHT,6,2011-11-30 17:37:00,2.08,15444,United Kingdom,2011-11-30,2011,12.48,2011-11
516367,579885,21257,VICTORIAN SEWING BOX MEDIUM,1,2011-11-30 17:37:00,7.95,15444,United Kingdom,2011-11-30,2011,7.95,2011-11


- 월별 고유한 날짜리스트를 만듬

In [52]:
month_list = sorted(retention_base['date_ym'].unique().tolist())
month_list

[Period('2010-12', 'M'),
 Period('2011-01', 'M'),
 Period('2011-02', 'M'),
 Period('2011-03', 'M'),
 Period('2011-04', 'M'),
 Period('2011-05', 'M'),
 Period('2011-06', 'M'),
 Period('2011-07', 'M'),
 Period('2011-08', 'M'),
 Period('2011-09', 'M'),
 Period('2011-10', 'M'),
 Period('2011-11', 'M')]

- 리텐션을 계산할 것임
- 시작월 & 타겟월의 구매자의 unique한 값 구하고 교집합을 구할 것임
- customerID,date_ym,invoiceNo의 Unique한 값만 남기기

In [53]:
retention_base = retention_base.drop_duplicates(subset=['CustomerID','date_ym'],keep='first')
retention_base

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_ymd,year,amount,date_ym
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12-01,2010,15.30,2010-12
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,2010-12-01,2010,54.08,2010-12
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24,2010-12-01 08:45:00,3.75,12583,France,2010-12-01,2010,90.00,2010-12
46,536371,22086,PAPER CHAIN KIT 50'S CHRISTMAS,80,2010-12-01 09:00:00,2.55,13748,United Kingdom,2010-12-01,2010,204.00,2010-12
65,536374,21258,VICTORIAN SEWING BOX LARGE,32,2010-12-01 09:09:00,10.95,15100,United Kingdom,2010-12-01,2010,350.40,2010-12
...,...,...,...,...,...,...,...,...,...,...,...,...
516074,579865,23016,GLASS TWIST BON BON JAR,1,2011-11-30 16:39:00,7.85,16915,United Kingdom,2011-11-30,2011,7.85,2011-11
516083,579867,35970,ZINC FOLKART SLEIGH BELLS,12,2011-11-30 16:41:00,1.69,16265,United Kingdom,2011-11-30,2011,20.28,2011-11
516153,579872,23508,MINI PLAYING CARDS DOLLY GIRL,14,2011-11-30 16:54:00,0.42,14085,United Kingdom,2011-11-30,2011,5.88,2011-11
516190,579881,21616,4 PEAR BOTANICAL DINNER CANDLES,12,2011-11-30 17:22:00,1.25,12429,Denmark,2011-11-30,2011,15.00,2011-11


In [54]:
from tqdm.notebook import tqdm

In [55]:
retention_rate_df = pd.DataFrame()
for i in tqdm(month_list):
    for j in month_list:
        start = i
        end = j
        if start <= end:

            base = set(retention_base.query('date_ym == @start')['CustomerID'])
            target = set(retention_base.query('date_ym == @end')['CustomerID'])
            retention = base.intersection(target)
            retention_rate = len(retention)/len(base)
            retention_df = pd.DataFrame({'cohort':[start],'date_ym':[end],'retention_rate':[retention_rate]})
            retention_rate_df = pd.concat([retention_rate_df,retention_df],axis=0)

retention_rate_df

  0%|          | 0/12 [00:00<?, ?it/s]

,cohort,date_ym,retention_rate
0,2010-12,2010-12,1.000000
0,2010-12,2011-01,0.366102
0,2010-12,2011-02,0.323164
0,2010-12,2011-03,0.384181
0,2010-12,2011-04,0.362712
...,...,...,...
0,2011-09,2011-10,0.386256
0,2011-09,2011-11,0.522117
0,2011-10,2011-10,1.000000
0,2011-10,2011-11,0.456012


In [56]:
retention_rate_df['cohort_size'] = (retention_rate_df['date_ym'] - retention_rate_df['cohort']).apply(lambda x: x.n)
retention_rate_df[['cohort','date_ym']] = retention_rate_df[['cohort','date_ym']].apply(lambda x: x.astype('str'))

In [57]:
retention_final = pd.pivot_table(retention_rate_df,values='retention_rate',index='cohort',columns='cohort_size')
retention_final

cohort_size,0,1,2,3,4,5,6,7,8,9,10,11
cohort,,,,,,,,,,,,
2010-12,1.0,0.366102,0.323164,0.384181,0.362712,0.397740,0.362712,0.349153,0.353672,0.395480,0.374011,0.502825
2011-01,1.0,0.353576,0.430499,0.383266,0.457490,0.419703,0.391363,0.384615,0.438596,0.417004,0.514170,NaN
2011-02,1.0,0.382586,0.375989,0.468338,0.428760,0.405013,0.402375,0.440633,0.414248,0.485488,NaN,NaN
2011-03,1.0,0.312115,0.441478,0.401437,0.377823,0.360370,0.421971,0.390144,0.474333,NaN,NaN,NaN
2011-04,1.0,0.429907,0.419393,0.407710,0.387850,0.450935,0.417056,0.491822,NaN,NaN,NaN,NaN
2011-05,1.0,0.388258,0.407197,0.405303,0.442235,0.416667,0.501894,NaN,NaN,NaN,NaN,NaN
2011-06,1.0,0.368315,0.409687,0.467205,0.435923,0.517659,NaN,NaN,NaN,NaN,NaN,NaN
2011-07,1.0,0.408851,0.457323,0.457323,0.521602,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08,1.0,0.454545,0.449198,0.534759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
fig = px.imshow(retention_final,text_auto='.2%',color_continuous_scale='reds')
fig

## **리텐션 분석결과**
***
- 최근으로 올수록 리텐션 비율이 오르는 고무적인 모습
- 2011년 11월에 많은 고객이 복귀함

- 리텐션 커브(코호트 사이즈별 리텐션 비율의 평균을 내는 것)

In [59]:
retention_curve = retention_rate_df.groupby('cohort_size').agg({'retention_rate':'mean'})
retention_curve = retention_curve.reset_index(drop=False)

In [60]:
fig = px.line(data_frame=retention_curve,x='cohort_size',y='retention_rate')
fig.update_layout(title='코호트 사이즈별 리텐션 비율의 평균',paper_bgcolor='lightgray',plot_bgcolor='lightblue')
fig.update_yaxes(tickformat='.2%')
fig.update_traces(line=dict(color='red'))

# **RFM분석**
***

- RFM분석이란?
- Recency, Frequency, Monetary를 기준으로 고객을 유형화 하는 방법
    - Recency(최근성): 고객이 얼마나 최근에 구매 했는지
    - Frequency(빈도): 고객이 얼마나 자주 구매 했는지
    - Monetary(금액): 고객이 얼마를 지출했는지
- **고객유형을 세분화하여 맞춤형 전략을 구상할 수 있다.**
- **이번 분석에서는 Recency와 Monetary를 기반으로 RM 분석을 해볼 예정**

In [61]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date_ymd,year,amount,date_ym
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12-01,2010,15.30,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12-01,2010,22.00,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,2010,20.34,2010-12
...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,2011-12-09,2011,10.20,2011-12
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,2011-12-09,2011,12.60,2011-12
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,2011-12-09,2011,16.60,2011-12
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,2011-12-09,2011,16.60,2011-12


### **CustomerID별로 RM을 구하기**
***

In [77]:
rm_df = df.groupby('CustomerID',as_index=False).agg({'date_ymd':lambda x: max(x),
                              'amount':lambda x:sum(x)})
rm_df

,CustomerID,date_ymd,amount
0,12346,2011-01-18,77183.60
1,12347,2011-12-07,4310.00
2,12348,2011-09-25,1797.24
3,12349,2011-11-21,1757.55
4,12350,2011-02-02,334.40
...,...,...,...
4334,18280,2011-03-07,180.60
4335,18281,2011-06-12,80.82
4336,18282,2011-12-02,178.05
4337,18283,2011-12-06,2094.88


### **pd.qcut을 이용하여 Recency를 5등급으로 나누기**
***
    - pd.qcut은 각 n분위수에 기반하여 cut해주는 함수

### **Recency와 Monetary점수를 3등급으로 각각 나누기**
***

In [78]:
rm_df

,CustomerID,date_ymd,amount
0,12346,2011-01-18,77183.60
1,12347,2011-12-07,4310.00
2,12348,2011-09-25,1797.24
3,12349,2011-11-21,1757.55
4,12350,2011-02-02,334.40
...,...,...,...
4334,18280,2011-03-07,180.60
4335,18281,2011-06-12,80.82
4336,18282,2011-12-02,178.05
4337,18283,2011-12-06,2094.88


In [79]:
rm_df['Recency'] = pd.qcut(x=rm_df['date_ymd'],q=3,labels=['C','B','A'])
rm_df['Monetary'] = pd.qcut(x=rm_df['amount'],q=3,labels=['C','B','A'])
rm_df

,CustomerID,date_ymd,amount,Recency,Monetary
0,12346,2011-01-18,77183.60,C,A
1,12347,2011-12-07,4310.00,A,A
2,12348,2011-09-25,1797.24,B,A
3,12349,2011-11-21,1757.55,A,A
4,12350,2011-02-02,334.40,C,C
...,...,...,...,...,...
4334,18280,2011-03-07,180.60,C,C
4335,18281,2011-06-12,80.82,C,C
4336,18282,2011-12-02,178.05,A,C
4337,18283,2011-12-06,2094.88,A,A


### **Recency와 Monetary 스코어를 모두 표시한 칼럼 추가**
***

In [80]:
rm_df['RM_Score'] = rm_df['Recency'].astype('str') + rm_df['Monetary'].astype('str')
rm_df

,CustomerID,date_ymd,amount,Recency,Monetary,RM_Score
0,12346,2011-01-18,77183.60,C,A,CA
1,12347,2011-12-07,4310.00,A,A,AA
2,12348,2011-09-25,1797.24,B,A,BA
3,12349,2011-11-21,1757.55,A,A,AA
4,12350,2011-02-02,334.40,C,C,CC
...,...,...,...,...,...,...
4334,18280,2011-03-07,180.60,C,C,CC
4335,18281,2011-06-12,80.82,C,C,CC
4336,18282,2011-12-02,178.05,A,C,AC
4337,18283,2011-12-06,2094.88,A,A,AA


### **고객 등급을 Recency, Monetary에 기반하여 분류하기**
***

In [81]:
import numpy as np

In [84]:
rm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4339 entries, 0 to 4338
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   CustomerID  4339 non-null   object        
 1   date_ymd    4339 non-null   datetime64[ns]
 2   amount      4339 non-null   float64       
 3   Recency     4339 non-null   category      
 4   Monetary    4339 non-null   category      
 5   RM_Score    4339 non-null   object        
dtypes: category(2), datetime64[ns](1), float64(1), object(2)
memory usage: 144.5+ KB


### **각 등급별 unique한 고객의 수 보기**
***

In [83]:
rm_df.groupby('RM_Score')[['CustomerID']].nunique()

,CustomerID
RM_Score,
AA,800
AB,407
AC,235
BA,475
BB,554
BC,419
CA,171
CB,485
CC,793


In [89]:
rm_df['고객등급'] = np.where(rm_df['RM_Score'] == 'AA','최우수고객', #Recency와 Monetary 모두 매우 높음
                         np.where((rm_df['RM_Score'] == 'AB')|(rm_df['RM_Score'] == 'BA')|(rm_df['RM_Score'] == 'BB'),'우수고객', #Recency와 Monetary 모두 높은 편
                         np.where((rm_df['RM_Score'] == 'BC')|(rm_df['RM_Score'] == 'AC'),'구매유도', #Recency는 높으나 Monetary가 낮아서 높은 가격의 프리미엄 상품의 구매를 더 유도해야함
                         np.where((rm_df['RM_Score'] == 'CA')|(rm_df['RM_Score'] == 'CB'),'이탈방지', #Monetary는 높으나 Recency가 낮아서 복귀유도를 해야함
                                   '휴면')))) # Recency와 Monetary 모두 낮아 휴면상태인 고객들  Recency와 Monetary모두 높여야함
rm_df

,CustomerID,date_ymd,amount,Recency,Monetary,RM_Score,고객등급
0,12346,2011-01-18,77183.60,C,A,CA,이탈방지
1,12347,2011-12-07,4310.00,A,A,AA,최우수고객
2,12348,2011-09-25,1797.24,B,A,BA,우수고객
3,12349,2011-11-21,1757.55,A,A,AA,최우수고객
4,12350,2011-02-02,334.40,C,C,CC,휴면
...,...,...,...,...,...,...,...
4334,18280,2011-03-07,180.60,C,C,CC,휴면
4335,18281,2011-06-12,80.82,C,C,CC,휴면
4336,18282,2011-12-02,178.05,A,C,AC,구매유도
4337,18283,2011-12-06,2094.88,A,A,AA,최우수고객


In [114]:
customer_grades = rm_df.groupby(by=['고객등급'],as_index=False)[['CustomerID']].nunique()
fig = px.funnel(data_frame=customer_grades,x='고객등급',y='CustomerID',color='CustomerID',
                labels={'CustomerID':'고객수'},title='등급별 고객수')
fig.update_layout(xaxis_title='고객등급',yaxis_title='고객수')
fig


c:\Users\kd010\miniconda3\envs\rapids\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



## RFM 분석 by 김동윤